Este proyecto tiene como objetivo estructurar una base de datos con información climática histórica proveniente de las estaciones de monitoreo de CONAGUA distribuidas en la República Mexicana.

Durante el desarrollo, identificamos dos retos técnicos principales:

- Estructuras DOM complejas y dinámicas: El sitio utiliza capas de diseño que impiden un web scraping convencional basado en peticiones estáticas.

- Acceso a datos históricos: La información de archivo no reside directamente en la interfaz, sino que se gestiona a través de redirecciones a archivos planos (.txt).

¿Suena complejo? Definitivamente lo fue. Para concretar este proyecto, fue necesario realizar un análisis de ingeniería inversa sobre la carga de etiquetas, automatizar la interacción con elementos dinámicos y diseñar una arquitectura de extracción que respete la integridad del sitio web fuente.

Este notebook documenta el proceso completo para ejecutar un web scraping masivo de manera eficiente.

Nota sobre el alcance: Aunque el sistema de CONAGUA ofrece múltiples variables, este proyecto se enfoca específicamente en la extracción de precipitaciones y temperaturas. Te invito a explorar el sitio fuente, donde cada marcador ("pin") contiene un catálogo aún más amplio de datos.

Ética y Responsabilidad
La ética en el scraping es innegociable. Para evitar la saturación de los servidores de CONAGUA, hemos implementado técnicas de navegación conservadoras, incluyendo intervalos de espera (time.sleep) calculados a conciencia.

Nota: Por razones de seguridad y buenas prácticas, ciertos segmentos sensibles del código (como endpoints específicos o credenciales) han sido omitidos o anonimizados.

Consideraciones adicionales:
- Deben de tener en cuenta que en ocasiones algunos censores no podran ser alcanzados. Para lidiar con esto se sugiere tener un manejo de errores que documente que censores atraves de su id no pudieron ser analizados, de esta manera mantienes una extraccion resiliente. 

![alt text](image.png)

In [ ]:
%pip install requests pandas selenium
import requests
import time
import random
import pandas as pd
import sys
# --- Librerías de Selenium (Completas y Ordenadas para entorno local) ---
from selenium import webdriver
from selenium.webdriver.common.by import By
# Importaciones específicas para Chrome (mejor compatibilidad con versiones recientes)
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException, WebDriverException

In [ ]:
## Ejecucion de pruebas 200 para acreditar el acceso a los links.
import requests
import time
import random
# import csv # Ya no es necesario
# import os  # Ya no es necesario
import pandas as pd

# Definir directamente la URL a probar
urls = ["https://smn.conagua.gob.mx/tools/GUI/ENCS.php?logo=0"]

resultados = []

print("Iniciando prueba de conexión...")

# Iterar sobre las URLs y probar cada una. En este caso estamos probando una sola URL.
for i, url in enumerate(urls):
    print(f"Probando link número {i+1} de {len(urls)}: {url}")
    try:
        user_agents = [
    """ 
    Es necesario la inclusion de webdrivers para emular un entorno en mozila firefox o google chrome
    Se recomienda Mozila
    """,
        ]
        headers = {'User-Agent': random.choice(user_agents)}

        # Realiza la solicitud
        response = requests.get(url, headers=headers, timeout=10)

        # Levanta un error para códigos 4xx/5xx
        response.raise_for_status()

        codigo_estado = response.status_code
        error = ""
    except requests.exceptions.RequestException as e:
        # Intenta obtener el código si la respuesta existe, si no, usa un mensaje genérico.
        codigo_estado = f"Error {response.status_code}" if 'response' in locals() and response is not None and response.status_code is not None else "Error de Conexión"
        error = str(e)
        print(f"Error con {url}: {e}")
    except Exception as e:
        # Captura cualquier otro error inesperado
        codigo_estado = "Error Inesperado"
        error = str(e)
        print(f"Error inesperado con {url}: {e}")
    finally:
        # Registra el resultado en la lista
        resultados.append([i+1, url, codigo_estado, error])

    time.sleep(random.uniform(5, 10))

print("\n--- Pruebas completadas ---")

# Crear DataFrame y visualizarlo
columnas = ["Número de Link", "URL", "Código de Estado", "Error"]
df_resultados = pd.DataFrame(resultados, columns=columnas)

df_resultados

El codigo 200 acredita exitosamente nuestra solicitud de conexion.

Metodología de Extracción Dinámica
Para evitar procesos redundantes y consolidar la navegación de Selenium, se diseñó una estrategia de interacción que garantiza el acceso a los elementos del DOM dinámico. Dado que la información climática solo se genera tras eventos específicos del usuario, el código implementa un flujo de trabajo basado en cuatro pilares:

1. Preparación del Entorno (Setup de Interfaz)
- Antes de buscar un sensor específico, el script automatiza la configuración del mapa para asegurar que la información sea "clicable":

- Gestión de Capas: Se activan tanto las estaciones "operando" como las "suspendidas" mediante selectores CSS, garantizando que el universo de datos sea completo.

- Ajuste de Visibilidad: Se implementa un control de Zoom In y el cierre de paneles laterales para maximizar el área de interacción y evitar errores de superposición de elementos (MoveTargetOutOfBoundsException).

2. Orquestación de Eventos Complejos (ActionChains)
- La información objetivo no reside de forma estática en el HTML. Para extraerla, el script imita el comportamiento humano avanzado:

- Doble Clic Estratégico: Se utiliza la clase ActionChains para ejecutar un doble clic sobre el marcador del sensor. Este evento es el disparador necesario para que el sitio despliegue y "ancle" el contenedor de información (popover-content).

- Limpieza de Estado: Tras cada extracción, el script realiza un clic en una zona neutra del mapa para cerrar ventanas emergentes y preparar la interfaz para la siguiente búsqueda.

3. Sincronización y Esperas Inteligentes
- Para mitigar la latencia del sitio y no saturar sus servidores, se emplean dos tipos de control temporal:

- Esperas Explícitas (WebDriverWait): El código espera hasta 45 segundos a que elementos cruciales aparezcan, permitiendo que el script sea resiliente a conexiones lentas.

- Intervalos de Estabilidad (time.sleep): Pausas estratégicas de 1 a 3 segundos que dan margen a las animaciones de la interfaz y previenen bloqueos por comportamiento automatizado agresivo.

4. Extracción y Limpieza con RegEx
- Una vez que el contenido dinámico es visible, se recupera el atributo innerHTML. Dado que los datos vienen en bloques de texto mezclados con etiquetas HTML, se utiliza Expresiones Regulares (RegEx) para parsear con precisión:

- Nombre de la estación.

- Coordenadas geográficas (Latitud y Longitud).

- URLs directas a los archivos históricos (.txt).

In [ ]:
#Pruebas preeliminares exitosas
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException, StaleElementReferenceException, MoveTargetOutOfBoundsException
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.webdriver.firefox.options import Options as FirefoxOptions 
import time
import sys
import pandas as pd
import re 
from pprint import pprint 

# -------------------------------------------------------------------
# --- PARTE 1: CONFIGURACIÓN Y SELECTORES ---
# -------------------------------------------------------------------
URL_BASE = "https://smn.conagua.gob.mx"
URL_CONAGUA = f"{URL_BASE}/tools/GUI/ENCS.php?logo=0"
WAIT_TIME = 45 # Espera máxima para elementos cruciales

ID_A_PROCESAR = ["1001"] 

# Selectores CSS/XPath
####

# XPaths Robustos: Buscan los enlaces por el texto que contienen
###

resultados_finales = []
driver = None

# -------------------------------------------------------------------
# --- FUNCIÓN DE EXTRACCIÓN POR SENSOR (DOBLE CLIC IMPLEMENTADO) ---
# -------------------------------------------------------------------

def extraer_datos_sensor(id_sensor, driver):
    """Realiza la búsqueda, el evento de doble clic y extrae los datos de un sensor."""
    #Datos a extaer
    datos_sensor = {
        'Nombre': str(id_sensor), 
        'Nombre': None,
        'Latitud': None,
        'Longitud': None,
        'URL_Diaria': None,
        'URL_Mensual': None,
        'Estado': 'FALLO'
    }
    
    # ----------------------- FASE DE BÚSQUEDA -----------------------
    try:
        CADENA_DE_BUSQUEDA = f"- {id_sensor}"
        print(f"\n[PROCESANDO] ID: {id_sensor} | Búsqueda: {CADENA_DE_BUSQUEDA}")
        
        search_box = WebDriverWait(driver, WAIT_TIME).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, css_selector_search_box))
        )
        
        search_box.clear() 
        search_box.send_keys(CADENA_DE_BUSQUEDA)
        time.sleep(1.5) 
        search_box.send_keys(Keys.ENTER)
        time.sleep(3) 
        
    except Exception as e:
        print(f" [ERROR] Fallo en la búsqueda del sensor {id_sensor}. Error: {type(e).__name__}. Saltando.")
        return datos_sensor

    # ----------------------- FASE DE DOBLE CLIC PARA DESPLEGAR CONTENIDO -----------------------
    try:
        # Esperar a que el pop-up ancla sea visible para el usuario (prevención de MoveTargetOutOfBoundsException)
        popup_ancla = WebDriverWait(driver, WAIT_TIME).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, css_selector_popup_ancla))
        )
        
        # Pausa extra para estabilidad
        time.sleep(1) 
        
        # NUEVA ACCIÓN CLAVE: Simular el doble clic 
        print(" [DOBLE CLIC ACTIONCHAINS] Ejecutando doble_click en div#popup para desplegar y anclar contenido...")
        ActionChains(driver).double_click(popup_ancla).perform()
        
        time.sleep(1)
        
        # Esperar a que el contenedor principal del pop-up esté visible después del doble clic
        popover_content = WebDriverWait(driver, WAIT_TIME).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, css_selector_popover_content))
        )
        print(" [DIAGNÓSTICO] Contenedor popover-content es visible. Iniciando extracción...")
        
    except MoveTargetOutOfBoundsException as e:
        print(f" [ERROR] MoveTargetOutOfBoundsException: El ancla del pop-up no estaba en la pantalla/interactuable. Detalles: {e.msg.splitlines()[0]}")
        return datos_sensor
    except Exception as e:
        print(f" [ERROR] Fallo al hacer doble clic o al encontrar el contenedor popover-content. Error: {type(e).__name__} - {e}")
        return datos_sensor

    # ----------------------- FASE DE EXTRACCIÓN DE DATOS -----------------------
    
    # 1. Recuperar la celda con datos generales (TD)
    try:
        print(f" [SINCRONIZACIÓN] Esperando {WAIT_TIME} segundos por el contenido de la tabla...")
        datos_generales_td = WebDriverWait(driver, WAIT_TIME).until(
             EC.presence_of_element_located((By.CSS_SELECTOR, css_selector_data_td))
        )
        datos_texto = datos_generales_td.get_attribute('innerHTML')
        print(" [EXTRACCIÓN] OK: Celda de datos generales (TD) encontrada.")
    except TimeoutException:
        print(f" [ERROR 1.1] TIMEOUT: La celda de datos no apareció después de {WAIT_TIME} segundos. La carga es demasiado lenta.")
        return datos_sensor
    except Exception as e:
        print(f" [ERROR 1.1] EXCEPCIÓN: Fallo al encontrar la CELDA de datos generales. Tipo: {type(e).__name__}. Detalles: {e}")
        return datos_sensor

    # 2. Extracción de URLs (Diaria y Mensual)
    try:
        popover_content_fresh = driver.find_element(By.CSS_SELECTOR, css_selector_popover_content)

        enlace_diaria = popover_content_fresh.find_element(By.XPATH, xpath_diaria)
        enlace_mensual = popover_content_fresh.find_element(By.XPATH, xpath_mensual)

        # Usamos directamente el valor del href (CORRECCIÓN PREVIA para evitar duplicidad de URL)
        datos_sensor['URL_Diaria'] = enlace_diaria.get_attribute("href")
        datos_sensor['URL_Mensual'] = enlace_mensual.get_attribute("href")
        
        print(" [EXTRACCIÓN] OK: Enlaces 'Diaria' y 'Mensual' encontrados y capturados.")

    except NoSuchElementException:
        print(f" [ERROR 1.2] NoSuchElementException: Uno o ambos enlaces (Diaria/Mensual) no fueron encontrados dentro del popover activo.")
        return datos_sensor
    except Exception as e:
        print(f" [ERROR 1.2] EXCEPCIÓN: Fallo al encontrar los enlaces. Tipo: {type(e).__name__}. Detalles: {e}")
        return datos_sensor

    # 3. Extracción de datos con RegEx
    try:
        nombre_match = re.search(r'<b>Nombre: </b>(.*?)<br>', datos_texto)
        latitud_match = re.search(r'<b>Latitud: </b>(.*?)º', datos_texto)
        longitud_match = re.search(r'<b>Longitud: </b>(.*?)º', datos_texto)
        
        if nombre_match:
            datos_sensor['Nombre'] = nombre_match.group(1).strip()
        if latitud_match:
            datos_sensor['Latitud'] = latitud_match.group(1).strip()
        if longitud_match:
            datos_sensor['Longitud'] = longitud_match.group(1).strip()
        
        datos_sensor['Estado'] = 'EXITO'
        print(" [EXTRACCIÓN] OK: Datos de RegEx procesados con éxito.")

    except Exception as e:
        print(f" [ERROR 1.3] EXCEPCIÓN: Fallo en el procesamiento RegEx. Tipo: {type(e).__name__}. Detalles: {e}")
        datos_sensor['Estado'] = 'FALLO (RegEx)'
        
    return datos_sensor

# -------------------------------------------------------------------
# --- PARTE 2: PRUEBA DE EJECUCIÓN ---
# -------------------------------------------------------------------
try:
    # 2.1 Inicialización del driver
    print(" [INICIO] Intentando inicializar Mozilla Firefox (GeckoDriver)...")
    driver = webdriver.Firefox() 
    driver.get(URL_CONAGUA)
    
    # 2.2 Configuración inicial (Pasos 1, 2 y 3)
    arrow_icon = WebDriverWait(driver, WAIT_TIME).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector_arrow)))
    arrow_icon.click()
    
    time.sleep(2)
    
    for selector in [css_selector_checkbox_operando, css_selector_checkbox_suspendida]:
        checkbox = WebDriverWait(driver, WAIT_TIME).until(EC.element_to_be_clickable((By.CSS_SELECTOR, selector)))
        if not checkbox.is_selected(): checkbox.click() 
    time.sleep(1) 
    
    # Zoom In 
    print(f"\n[PASO 5.1] Dando doble clic de Zoom In...")
    zoom_in_button = WebDriverWait(driver, WAIT_TIME).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector_zoom_in))
    )
    # Doble clic para asegurar un buen nivel de zoom 
    zoom_in_button.click()
    time.sleep(1) 
    zoom_in_button.click() # Segundo clic para zoom adicional
    time.sleep(1)
    
    # 2.3 Cerrar Panel (Paso 5.2)
    close_button = WebDriverWait(driver, WAIT_TIME).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector_close_panel)))
    close_button.click()
    time.sleep(1)

    # 2.4 Bucle de prueba (solo ID 1001)
    print("\n--- INICIANDO PRUEBA CON SENSOR 1001 ---")
    for id_sensor in ID_A_PROCESAR:
        datos = extraer_datos_sensor(id_sensor, driver)
        resultados_finales.append(datos)
        
        # Clic simple en el mapa para cerrar el pop-up y limpiar para la próxima búsqueda
        map_element = driver.find_element(By.ID, "map") 
        ActionChains(driver).move_to_element(map_element).click().perform()
        time.sleep(1)


finally:
    # -------------------------------------------------------------------
    # --- RESULTADOS DE LA PRUEBA ---
    # -------------------------------------------------------------------
    print("\n==============================================")
    print("  RESULTADO DE LA PRUEBA (SENSOR 1001)  ")
    print("==============================================")
    
    if resultados_finales:
        df_prueba = pd.DataFrame(resultados_finales)
        print(df_prueba.to_string(index=False)) 
    else:
        print("La extracción para el sensor 1001 falló.") #manejo de errores, muy importante
        
    if driver:
        print("\nPrueba finalizada. Cierra el navegador manualmente.")

Pueden encontrar estos csv en el sitio de CONAGUA afiliado al visual sobre el cual se está trabajando. 

In [ ]:
#Csv de las bases de datos

operativos = pd.read_csv(r'direccion_x....\operativas.csv')
suspendidos = pd.read_csv(r'direccion_x....\suspendidas.csv')
operativos

Implementación del Procesamiento Masivo y Consolidación de Datos
Tras validar la técnica de interacción dinámica, el siguiente paso consiste en escalar la extracción a nivel nacional. Esta sección del código implementa una arquitectura de procesamiento por lotes (batch processing) que integra fuentes de datos externas y mecanismos de persistencia para consolidar la base de datos climática.

La estrategia se divide en tres componentes clave:

1. Ingesta y Unificación de Catálogos
El proceso comienza fuera del navegador. El script utiliza la librería pandas para leer catálogos locales de estaciones:

- Fusión de Estructuras: Se cargan y unifican los listados de estaciones operativas y suspendidas.

- Normalización de IDs: Se extraen y limpian los identificadores únicos (columna Name), eliminando duplicados mediante estructuras de conjuntos (set) para asegurar que cada torre climatológica se procese exactamente una vez.

2. Ciclo de Extracción de Alto Rendimiento
Para procesar cientos de estaciones de forma consecutiva sin que el script colapse, se refinó la función de extracción con controles de estado:

- Sincronización de Contenido Crítico: A diferencia de una carga web normal, aquí el script espera a que el enlace interno "Climatología diaria" sea detectable en el DOM antes de proceder. Esto actúa como un ancla de seguridad que confirma que el pop-up no está vacío.

- Recuperación Automática de Errores: Cada sensor se envuelve en bloques try-except. Si una estación falla (por carga lenta o error de red), el script registra el estado de FALLO, limpia la interfaz y continúa con la siguiente, evitando la interrupción total del proceso masivo.

3. Persistencia y Auditoría de Resultados
- El objetivo final es transformar la interacción visual en datos estructurados listos para el análisis:

- Extracción Multivariable: Se capturan simultáneamente metadatos geográficos (Latitud/Longitud) y los puntos de acceso (URLs) a los archivos .txt históricos.

- Exportación Robusta: Al finalizar el bucle, los resultados se consolidan en un DataFrame maestro que se exporta a un archivo CSV mediante rutas de cadena cruda (raw strings) para garantizar la compatibilidad con el sistema de archivos de Windows.

- Resumen de Operación: El sistema genera un reporte automático de estados (Value Counts) que permite auditar cuántas estaciones fueron procesadas con éxito y cuántas requieren una revisión manual.

- Seguridad de la Información: Se han implementado rutas de acceso y selectores protegidos para cumplir con las mejores prácticas de seguridad, asegurando que el flujo lógico sea reproducible sin exponer información sensible del entorno local.

In [ ]:
#Extraccion de los archivos .txt de las torres climatologicas al nivel nacional 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException, StaleElementReferenceException, MoveTargetOutOfBoundsException
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.webdriver.firefox.options import Options as FirefoxOptions 

import time
import sys
import pandas as pd
import re 
from pprint import pprint 
import os 

# -------------------------------------------------------------------
# --- PARTE 1: CONFIGURACIÓN Y DATOS DE ENTRADA (CARGA MEJORADA) ---
# -------------------------------------------------------------------
URL_BASE = "https://smn.conagua.gob.mx"
URL_CONAGUA = f"{URL_BASE}/tools/GUI/ENCS.php?logo=0"
WAIT_TIME = 45 # Espera máxima para elementos cruciales

# Lectura directa de archivos CSV (Tu implementación, asegurando la ruta 'raw' con 'r')
try:
    operativos = pd.read_csv(r'direccion_x....\operativas.csv')
    suspendidos = pd.read_csv(r'direccion_x....\suspendidas.csv')

    # CORRECCIÓN CRÍTICA 1: Usar 'r' (raw string) para la ruta de salida para evitar errores de escape.
    ARCHIVO_SALIDA = r"direccion_x...\Resultados_conagua.csv"
    
    # Asignamos los DataFrames para usarlos en la función de carga (adaptación)
    ARCHIVO_OPERATIVAS_DF = operativos
    ARCHIVO_SUSPENDIDAS_DF = suspendidos
    
except FileNotFoundError as e:
    print(f"\n[ERROR FATAL] Uno o ambos archivos CSV no se encontraron. Revise la ruta.")
    print(f"Detalle del error: {e}")
    sys.exit()
except Exception as e:
    print(f"\n[ERROR FATAL] Ocurrió un error al leer los archivos CSV: {e}")
    sys.exit()


# Selectores CSS/XPath (Se mantienen sin cambios)
# selectores censurados

# XPaths Robustos (Se mantienen sin cambios)
# XPaths censurados

resultados_finales = []
driver = None

# -------------------------------------------------------------------
# --- FUNCIÓN DE CARGA Y PREPARACIÓN DE DATOS (COLUMNA 'Name' CORREGIDA) ---
# -------------------------------------------------------------------

def cargar_ids_a_procesar(df_operativas, df_suspendidas):
    """Extrae y combina los IDs únicos de las estaciones desde los DataFrames usando la columna 'Name'."""
    
    print("[CARGA DATOS] Extrayendo IDs de los DataFrames cargados (usando columna 'Name')...")

    try:

        ids_operativas = df_operativas['Name'].dropna().astype(str).tolist()
        print(f" [OK] {len(ids_operativas)} IDs Operativas cargadas.")
        

        ids_suspendidas = df_suspendidas['Name'].dropna().astype(str).tolist()
        print(f" [OK] {len(ids_suspendidas)} IDs Suspendidas cargadas.")
        
    except KeyError as e:
        print(f" [ERROR] Error: El DataFrame no contiene una columna llamada 'Name'. Revise el archivo CSV. Detalle: {e}")
        return []
    except Exception as e:
        print(f" [ERROR] Fallo al procesar los IDs. Error: {e}")
        return []

    # Combinar y retornar
    ids_combinados = list(set(ids_operativas + ids_suspendidas))
    print(f" [RESUMEN] Total de IDs únicos a procesar: {len(ids_combinados)}")
    
    return ids_combinados

# -------------------------------------------------------------------
# --- FUNCIÓN DE EXTRACCIÓN POR SENSOR  ---
# -------------------------------------------------------------------

def extraer_datos_sensor(id_sensor, driver):
    """Realiza la búsqueda, el evento de doble clic y extrae los datos de un sensor."""
    
    #Datos a extaer
    datos_sensor = {
        'Clave': str(id_sensor), 
        'Nombre': None,
        'Latitud': None,
        'Longitud': None,
        'URL_Diaria': None,
        'URL_Mensual': None,
        'Estado': 'FALLO'
    }
    
    # ----------------------- FASE DE BÚSQUEDA -----------------------
    try:
        # La búsqueda utiliza el ID (que ahora es el valor de la columna 'Name')
        CADENA_DE_BUSQUEDA = f"- {id_sensor}" 
        print(f"\n[PROCESANDO] ID: {id_sensor} | Búsqueda: {CADENA_DE_BUSQUEDA}")
        
        search_box = WebDriverWait(driver, WAIT_TIME).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, css_selector_search_box))
        )
        
        search_box.clear() 
        search_box.send_keys(CADENA_DE_BUSQUEDA)
        time.sleep(1.5) 
        search_box.send_keys(Keys.ENTER)
        time.sleep(3) 
        
    except Exception as e:
        print(f" [ERROR] Fallo en la búsqueda del sensor {id_sensor}. Error: {type(e).__name__}.")
        return datos_sensor

    # ----------------------- FASE DE DOBLE CLIC PARA DESPLEGAR CONTENIDO -----------------------
    try:
        popup_ancla = WebDriverWait(driver, WAIT_TIME).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, css_selector_popup_ancla))
        )
        
        time.sleep(1) 
        
        print(" [DOBLE CLIC ACTIONCHAINS] Ejecutando doble_click...")
        ActionChains(driver).double_click(popup_ancla).perform()
        
        time.sleep(1)
        
        # Espera que el contenedor principal del pop-up sea visible
        popover_content = WebDriverWait(driver, WAIT_TIME).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, css_selector_popover_content))
        )
        print(" [DIAGNÓSTICO] Contenedor popover-content visible.")
        
    except MoveTargetOutOfBoundsException as e:
        print(f" [ERROR] MoveTargetOutOfBoundsException: El pop-up no estaba en la pantalla/interactuable.")
        return datos_sensor
    except Exception as e:
        print(f" [ERROR] Fallo al hacer doble clic o al encontrar el popover. Error: {type(e).__name__}.")
        return datos_sensor

    # ----------------------- FASE DE EXTRACCIÓN DE DATOS -----------------------
    
    # CORRECCIÓN DE ESTABILIDAD: Esperar explícitamente por el enlace interno.
    try:
        print(" [SINCRONIZACIÓN] Esperando a que el enlace 'Climatología diaria' esté presente...")
        enlace_diaria = WebDriverWait(popover_content, WAIT_TIME).until(
            EC.presence_of_element_located((By.XPATH, xpath_diaria))
        )
        print(" [EXTRACCIÓN] OK: Enlace 'Diaria' encontrado. El pop-up está cargado.")
        
        # 1. Recuperar la celda con datos generales (TD)
        datos_generales_td = driver.find_element(By.CSS_SELECTOR, css_selector_data_td)
        datos_texto = datos_generales_td.get_attribute('innerHTML')

        # 2. Extracción de URLs (Mensual)
        # Reutilizamos el 'enlace_diaria' encontrado, y buscamos el 'enlace_mensual' desde el mismo contenedor padre.
        popover_content_fresh = driver.find_element(By.CSS_SELECTOR, css_selector_popover_content)
        enlace_mensual = popover_content_fresh.find_element(By.XPATH, xpath_mensual)

        datos_sensor['URL_Diaria'] = enlace_diaria.get_attribute("href")
        datos_sensor['URL_Mensual'] = enlace_mensual.get_attribute("href")
        print(" [EXTRACCIÓN] OK: Enlaces 'Diaria' y 'Mensual' capturados.")
        
    except TimeoutException:
        print(f" [ERROR 1.1] TIMEOUT: El contenido del pop-up no apareció después de {WAIT_TIME} segundos.")
        return datos_sensor
    except Exception as e:
        print(f" [ERROR 1.2] EXCEPCIÓN: Fallo al encontrar la CELDA o los enlaces. Tipo: {type(e).__name__}.")
        return datos_sensor

    # 3. Extracción de datos con RegEx (Sin cambios)
    try:
        # Se asume que el ID que buscas es el que aparece como 'Clave' en el resultado final
        datos_sensor['Clave'] = str(id_sensor).strip() 
        
        # Las extracciones de Nombre, Latitud y Longitud se mantienen usando el texto del pop-up
        nombre_match = re.search(r'<b>Nombre: </b>(.*?)<br>', datos_texto)
        latitud_match = re.search(r'<b>Latitud: </b>(.*?)º', datos_texto)
        longitud_match = re.search(r'<b>Longitud: </b>(.*?)º', datos_texto)
        
        if nombre_match:
            # En este caso, el nombre de la estación se extrae del texto del pop-up, no del CSV.
            datos_sensor['Nombre'] = nombre_match.group(1).strip()
        if latitud_match:
            datos_sensor['Latitud'] = latitud_match.group(1).strip()
        if longitud_match:
            datos_sensor['Longitud'] = longitud_match.group(1).strip()
        
        datos_sensor['Estado'] = 'EXITO'
        print(" [RESULTADO] Extracción de datos exitosa.")
    except Exception as e:
        print(f" [ERROR 1.3] EXCEPCIÓN: Fallo en el procesamiento RegEx. Tipo: {type(e).__name__}.")
        datos_sensor['Estado'] = 'FALLO (RegEx)'
        
    return datos_sensor

# -------------------------------------------------------------------
# --- PARTE 2: PRUEBA DE EJECUCIÓN MASIVA ---
# -------------------------------------------------------------------

# LLAMADA CORREGIDA: Pasamos los DataFrames cargados globalmente
ID_A_PROCESAR = cargar_ids_a_procesar(ARCHIVO_OPERATIVAS_DF, ARCHIVO_SUSPENDIDAS_DF)

# Solo se procede si se cargaron IDs
if ID_A_PROCESAR:
    try:
        # 2.1 Inicialización del driver
        print(" [INICIO] Intentando inicializar Mozilla Firefox (GeckoDriver)...")
        driver = webdriver.Firefox() 
        driver.get(URL_CONAGUA)
        
        # 2.2 Configuración inicial (filtros y doble zoom)
        print(" [SETUP] Configurando el mapa (filtros y zoom)...")
        arrow_icon = WebDriverWait(driver, WAIT_TIME).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector_arrow)))
        arrow_icon.click()
        time.sleep(2)
        
        for selector in [css_selector_checkbox_operando, css_selector_checkbox_suspendida]:
            checkbox = WebDriverWait(driver, WAIT_TIME).until(EC.element_to_be_clickable((By.CSS_SELECTOR, selector)))
            if not checkbox.is_selected(): checkbox.click() 
        time.sleep(1) 
        
        # Doble Zoom In 
        zoom_in_button = WebDriverWait(driver, WAIT_TIME).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector_zoom_in))
        )
        zoom_in_button.click()
        time.sleep(1) 
        zoom_in_button.click() # Segundo clic para zoom adicional
        time.sleep(1)
        
        # 2.3 Cerrar Panel
        close_button = WebDriverWait(driver, WAIT_TIME).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector_close_panel)))
        close_button.click()
        time.sleep(1)

        # 2.4 Bucle de procesamiento masivo
        print("\n--- INICIANDO PROCESAMIENTO MASIVO DE ESTACIONES ---")
        total_ids = len(ID_A_PROCESAR)
        
        for i, id_sensor in enumerate(ID_A_PROCESAR):
            print(f"\n[PROGRESO] Estación {i+1} de {total_ids}")
            
            datos = extraer_datos_sensor(id_sensor, driver)
            resultados_finales.append(datos)
            
            # Limpiar el pop-up
            try:
                map_element = driver.find_element(By.ID, "map") 
                ActionChains(driver).move_to_element(map_element).click().perform()
                time.sleep(1)
            except Exception as e:
                print(f" [ADVERTENCIA] No se pudo limpiar el pop-up. Error: {type(e).__name__}.")


    finally:
        # -------------------------------------------------------------------
        # --- RESULTADOS FINALES Y ALMACENAMIENTO ---
        # -------------------------------------------------------------------
        print("\n==============================================")
        print("      PROCESAMIENTO FINALIZADO      ")
        print("==============================================")
        
        if resultados_finales:
            df_final = pd.DataFrame(resultados_finales)
            
            # Almacenar los resultados en un CSV
            try:
                df_final.to_csv(ARCHIVO_SALIDA, index=False, encoding='utf-8') #Guardando resultados
                print(f" [CSV EXPORTADO] Resultados guardados en '{ARCHIVO_SALIDA}'.")
                print(f"\nResumen de resultados:\n{df_final['Estado'].value_counts().to_string()}")
            except Exception as e:
                print(f" [ERROR FATAL] No se pudo guardar el archivo CSV '{ARCHIVO_SALIDA}'. Error: {e}")
        else:
            print("La extracción falló completamente. No se obtuvieron resultados.")
            
        if driver:
            print("\nPrueba finalizada. Cierra el navegador manualmente.")

else:
    print("\n==============================================")
    print("      PROCESAMIENTO ABORTADO      ")
    print("==============================================")
    print("No se cargó ningún ID. Por favor, asegúrese de que la ruta de los archivos CSV sea correcta y que contengan la columna 'Name'.")